In [ ]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta,time
from streamlit_gsheets import GSheetsConnection
from auxiliar.auxiliar import *
from auxiliar.omie_aux import *

In [ ]:
servicos = atualizar_servicos()
